In [ ]:
%%file gridding.py

from mpi4py import MPI
import numpy as np
import pandas as pd
from scipy.fft import ifft2
import sys

# Función para transformar coordenadas u y v a longitudes de onda
def transform_coordinates(data, frequency):
    c = 299792458.0  # Velocidad de la luz en metros por segundo
    u_lambda = data[:, 0] * (frequency / c)
    v_lambda = data[:, 1] * (frequency / c)
    return u_lambda, v_lambda

# Función para realizar el proceso de gridding en cada proceso
def grid_data(local_data, u_lambda, v_lambda, N, delta_x_rad):
    # Matrices para F_r, F_i y W_t locales
    local_F_r = np.zeros((N, N), dtype=np.float64)
    local_F_i = np.zeros((N, N), dtype=np.float64)
    local_W_t = np.zeros((N, N), dtype=np.float64)

    weight = local_data[:, 5]  # Peso

    # Calcular ik y jk redondeando solo la parte de (u_lambda_k / delta_x_rad) y (v_lambda_k / delta_x_rad)
    ik = np.round(u_lambda / delta_x_rad).astype(int) + N // 2
    jk = np.round(v_lambda / delta_x_rad).astype(int) + N // 2

    # Filtrar valores válidos
    valid_indices = (ik >= 0) & (ik < N) & (jk >= 0) & (jk < N)

    # Acumular los valores en las matrices locales
    local_F_r[ik[valid_indices], jk[valid_indices]] += weight[valid_indices] * local_data[valid_indices, 3]
    local_F_i[ik[valid_indices], jk[valid_indices]] += weight[valid_indices] * local_data[valid_indices, 4]
    local_W_t[ik[valid_indices], jk[valid_indices]] += weight[valid_indices]

    return local_F_r, local_F_i, local_W_t

# Inicializar MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Obtener argumentos de línea de comandos
if len(sys.argv) != 7:
    print("Uso: mpirun -n numeroprocesos python gridding.py -i datosuv.raw -d deltax -N tamañoimagen")
    sys.exit(1)

if sys.argv[1] != '-i':
    print("Error: Debes especificar el archivo de datos con -i")
    sys.exit(1)

if sys.argv[3] != '-d':
    print("Error: Debes especificar el valor de deltax con -d")
    sys.exit(1)

if sys.argv[5] != '-N':
    print("Error: Debes especificar el tamaño de la imagen con -N")
    sys.exit(1)

input_file = sys.argv[2]
deltax = float(sys.argv[4])
N = int(sys.argv[6])

if rank == 0:
    # Proceso con rank 0 lee los datos del archivo de entrada
    data = pd.read_csv(input_file, header=None, names=["uk", "vk", "wk", "real", "imag", "weight", "frequency", "channel"])
    data = data.values

    # Obtenemos la frecuencia de observación
    frequency = data[0, 6]

    # Transformamos las coordenadas u y v a longitudes de onda
    u_lambda, v_lambda = transform_coordinates(data, frequency)

    # Divide los datos en partes para cada proceso
    data_split = np.array_split(data, size, axis=0)
else:
    data_split = None

# Repartir los datos a todos los procesos
local_data = comm.scatter(data_split, root=0)

# Parámetros de la imagen y gridding
uv_size = 256  # Tamaño del plano uv
delta_x_arcsec = deltax  # Valor ∆x en arcsegundos
delta_x_rad = (np.pi / 180 / 3600) * delta_x_arcsec

# Realizar el proceso de gridding en cada proceso
local_F_r, local_F_i, local_W_t = grid_data(local_data, u_lambda, v_lambda, N, delta_x_rad)

if rank == 0:
    # Tamaño de datos que se espera recibir (en bytes)
    expected_data_size = N * N * (2 * np.float64().itemsize + np.float64().itemsize)

    # Crear un búfer lo suficientemente grande para acomodar los datos
    buffer_size = size * expected_data_size
    buffer = np.empty(buffer_size, dtype=np.uint8)

# Recolectar las matrices locales
comm.Gather(local_F_r, buffer, root=0)
comm.Gather(local_F_i, buffer, root=0)
comm.Gather(local_W_t, buffer, root=0)

if rank == 0:
    # Proceso con rank 0 suma todas las matrices
    global_F_r = np.zeros((N, N), dtype=np.float64)
    global_F_i = np.zeros((N, N), dtype=np.float64)
    global_W_t = np.zeros((N, N), dtype=np.float64)

    for i in range(size):
        start = i * expected_data_size
        end = start + expected_data_size

        # Recuperar datos de cada proceso
        local_F_r = buffer[start:end].view(dtype=np.float64).reshape((N, N))
        local_F_i = buffer[start + expected_data_size:end + expected_data_size].view(dtype=np.float64).reshape((N, N))
        local_W_t = buffer[start + 2 * expected_data_size:end + 2 * expected_data_size].view(dtype=np.float64).reshape((N, N))

        # Sumar a las matrices globales
        global_F_r += local_F_r
        global_F_i += local_F_i
        global_W_t += local_W_t

    # Verificar si W_t es igual a cero antes de la división
    non_zero_indices = global_W_t != 0

    # Dividir F_r y F_i solo en las celdas donde W_t no es cero
    global_F_r[non_zero_indices] /= global_W_t[non_zero_indices]
    global_F_i[non_zero_indices] /= global_W_t[non_zero_indices]

    # Realizar la Transformada de Fourier Inversa para obtener la imagen sucia
    dirty_image = np.abs(ifft2(global_F_r + 1j * global_F_i))

    # Mostrar la imagen sucia
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8, 8))
    plt.imshow(dirty_image, cmap='gray')
    plt.title('Imagen Sucia')
    plt.colorbar()
    plt.show()

In [ ]:
!mpirun -n 8 python gridding.py -i hltau_completo_uv.csv -d 0.1 -N 256